查看GPU

In [39]:
!nvidia-smi

Thu Apr 18 10:33:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              34W /  70W |   6637MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

安裝需要的BERT套件

In [40]:
!pip install transformers==4.29.0 -U
!pip install nlp -U
!pip install torch==2.0.0 -U
!pip install transformers accelerate

載入套件

In [41]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support



抓取訓練資料

In [42]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv

--2024-04-18 10:34:09--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-04-18 10:34:09--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>] 379.78K  --.-KB/s    in 0.02s   

2024-04-18 10:34:09 (18.7 MB/s) - ‘train.csv.1’ saved [388895/388895]



讀取訓練資料集

In [43]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df)


In [44]:
df.sample(10)


,ID,Name,Description,label
321,321,益妙兒,「鋅會干擾病毒picornavirus在體內的複製酵素，抑制病毒在體內複製；且可以減少病毒附...,1
70,70,視立保草本益明錠,「唯一能跟葉黃素抗衡的護眼營養素，是改善眼睛疲勞、改善視覺清晰度；增強睫狀肌調節力、預防單眼...,1
209,209,專品藥局淨邁舒PLUS加強版60粒紅葡萄葉萃取物、專品藥局淨邁舒PLUS加強版60粒紅葡萄葉...,「預防醫學…藥草」「維持泌尿道健康」「輕盈苗條…並佐以腰部曲線圖」「肝臟保健聖品」,1
656,656,全新生活?纖盈錠狀食品,全新生活?纖盈錠狀食品中主要含有植物性的成分白腎豆萃取物及LeptiCore?專利配方成分、...,0
685,685,植物營養纖滋棒(安麗)Nutritionbar,嘴饞又怕高熱量？以大豆、水果、堅果烘培而成的纖滋棒，高纖低卡又美味，無論正餐、點心或運動前後...,0
96,96,團圓堅果旺福財犬禮盒核桃/綜合堅果原味經典,「醫學期刊…降低血脂，有益心血管的健康」,1
468,468,【海洋傳奇】【日本出貨】FANCL芳珂體內脂肪SUPPORT30日份,「體內脂肪…平衡腸道環境…24週間…腹部脂肪面積…皮下脂肪面積減少…（佐以圖表呈現）」「保濕...,1
86,86,現貨!Bavaria德國連線德國原裝進口BIOPRAP兒童嬰幼兒藍莓葉黃素膠囊90粒,「護眼…服用」「淚膜修護…護眼…佐以眼睛示意圖」,1
532,532,長庚純正牛樟菇菌絲體純液搶購組,「…牛樟菇菌絲體防止肝臟發炎作用明顯增進肺癌抑制療效並緩化療藥物的肝臟毒性…牛樟菇含有…促進...,1
327,327,廣和月子餐,「【莊老師喜寶】...避免孕媽鈣質流失造成骨質疏鬆...廣和產前提供的【養肝茶可以幫助解胎毒...,1


載入原始模型

In [45]:

model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')



Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

載入分詞模型

In [46]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')

設定執行參數

In [47]:
RANDOM_SEED = 5
MAX_LEN = 512
EPOCHS=5
BATCH_SIZE = 8

把資料集分成訓練、測試、驗證

In [48]:

shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

取一筆出來瞧瞧(已經打亂)

In [49]:

train_dataset[2]

{'Description': '「香蕉抗性澱粉蕉纖盈(30包/盒x2盒)」「低GI…抗脂…-29…血糖升高…體重控制」',
 'ID': 138,
 'Name': '【小甜甜代言日本VANTEK】香蕉抗性澱粉蕉纖盈(30包/盒x2盒)',
 'label': 1}

In [50]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

定義分詞器

In [51]:
def tokenize(batch):
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

定義評測準則，訓練器

In [52]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

開始訓練

In [53]:

trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 0.08325294405221939,
 'eval_accuracy': 0.9893617021276596,
 'eval_f1': 0.9908256880733944,
 'eval_precision': 1.0,
 'eval_recall': 0.9818181818181818,
 'eval_runtime': 3.0299,
 'eval_samples_per_second': 31.024,
 'eval_steps_per_second': 3.96,
 'epoch': 5.0}

找一個範例來測試

In [54]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


設定測試文字

In [55]:
#test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'
test_text = '???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品'

MAX_LEN = 512

encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)


In [56]:
encoded_review

{'input_ids': tensor([[  101,   136,   136,   136,   136,   136,   136,   136,  3680,   819,
          1419,  3300,  8114,  8882,  6737, 11382,   511,  6737, 11382,  1377,
          1347,  5645,  5204,  4495,  5162,   147,  1086,  4495,  8024,  1060,
          5442,  1072,  1295,  1398,   868,  4500,  8024,  1377,   914,  6868,
          3173,  7376,   807,  6342,   511,   924,  6362,  2542,  4472,  7518,
          3268,  5204,  2898,   978,  2434,   511,  3268,  7077,  3189,  3315,
          4638,  3297,   881,  5401,  2159,  5469,  1501,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

進行測試

In [57]:
test_model = model.to('cuda')

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

classnames = ['合格', '不合格']

print(f'廣告詞: {test_text}')
print(f'檢測結果: {classnames[result]}')


廣告詞: ???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品
檢測結果: 合格
